# Tutorial by Marc LABADIE for software paper rebuild echap loop with adel protocol after conversion in python3

## 1. Adel:
Allows to simulate the 3D architectural development of the shoot of gramineaous plant. 
* Adel-Maize (Fournier & Andrieu, 1998)
* Adel-Wheat (Fournier et al. 2003) models

### 1.1 Create a wheat architectural model with geometry at a given development stage and visualize wheat

In [9]:
from alinea.adel.astk_interface import AdelWheat
import openalea.plantgl.all as pgl
from oawidgets.plantgl import PlantGL

%gui qt
#%pdb

In [ ]:
adel = AdelWheat(nplants=2) # load architectural rule and parameters of wheat
g = adel.setup_canopy(age=1500) # Select the age of wheat, age represent degree days
adel.plot(g)


### 1.2 Create an wheat architectural model at given developmental stage from data (Echap data)

In [10]:
from alinea.echap.architectural_reconstructions import EchapReconstructions

echap = EchapReconstructions() # load architectural parameter reconstruction with echap data

# load reconstruction of Mercia variaty at 1500 degree days
wheat_Mercia = echap.get_reconstruction(name="Mercia", nplants=1,seed=0)
g_Mercia= wheat_Mercia.setup_canopy(age=1500)
PlantGL(wheat_Mercia.plot(g_Mercia), group_by_color=False)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

In [ ]:
# load reconstruction of Tremie12 variaty at 1500 degree days
wheat_Tremie12 = echap.get_reconstruction(name="Tremie12", nplants=1,seed=0)
g_Tremie12= wheat_Tremie12.setup_canopy(age=1500)
PlantGL(wheat_Tremie12.plot(g_Tremie12), group_by_color=False)

### 1.3 It possible to simulate grow canopy using different age of the canopy or directely using weather data (temperature)

#### 1.3.1 Using canopy age:

* canopy_age initiat at 100 degree days
* grow_cycle: number of grow cycle
* grow_time: interval of degree days between each cycle (100)

In [ ]:
canopy_age= 100 # age of canopy at start
grow_cycles =20 # nb of growth cycle
grow_time = 100 #degree day, time interval between each cycle

wheat = echap.get_reconstruction(name="Tremie12", nplants=2,seed=0) # init a wheat mtg

for cycle in range(grow_cycles):
    canopy_age = canopy_age
    # update canopy
    new_g = wheat.setup_canopy(age=canopy_age)
    wheat.plot(new_g)
    canopy_age = canopy_age + grow_time

#### 1.3.2 With weather data issue from echap project

**selection of Weather parameter**

Based on weather data:
* weather_time_index: selection of the period where is applied simulation
* every_dd: among the period selected filter data by step of the thermal time here each 15 degree days 

In [ ]:
from alinea.echap.weather_data import get_weather

from alinea.alep.mini_models import linear_degree_days
from alinea.alep.protocol import update
from alinea.alep.growth_control import NoPriorityGrowthControl

from alinea.astk.TimeControl import DegreeDayModel, thermal_time_filter
from alinea.astk.TimeControl import IterWithDelays , time_control

import pandas

In [ ]:
# import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

#check weather data and add degree day variable using linear degree day model
weather.check(["degree_days",'temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure'],models={'degree_days':linear_degree_days})

# weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", end='2011-05-09', freq='H')  # Range date index. among weather data select weather period for simulation

# filter degree days
TTmodel = DegreeDayModel(Tbase = 0) # Thermal times model
every_dd = thermal_time_filter(time_sequence=weather_time_index, weather=weather, model=TTmodel, delay = 15) # allows to filter thermal time data here each 15 degree day
canopy_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_dd, data=weather.data)) # create data from weatherdata according to filter

# growth control
growth_controler = NoPriorityGrowthControl()

In [ ]:
wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g = wheat.setup_canopy(age=500)

for i,controls in enumerate(canopy_timing):
    canopy_iter = controls

    if canopy_iter.eval:
        print('--', canopy_iter.value.index[-1], '--')
        print('-- update microclimate / canopy --')
        g = wheat.grow(g, canopy_iter.value)
        _=update(g, canopy_iter.dt, growth_controler)
        wheat.plot(g)


## 2. Model Pesticide interception

In [11]:
from matplotlib import cm

In [19]:
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.interfaces import pesticide_interception
from alinea.echap.tests_nodes import plot_pesticide

In [14]:
application_data = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 100, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 100, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 100, Opus
2010-05-22 11:00:00, 0, bug
"""
pest_calendar = pesticide_applications(application_data)
pest_calendar

,date,dose,product_name
date,,,
2010-04-29 10:00:00,2010-04-29 10:00:00,100,Opus
2010-04-29 11:00:00,2010-04-29 11:00:00,0,bug
2010-05-22 10:00:00,2010-05-22 10:00:00,100,Opus
2010-05-22 11:00:00,2010-05-22 11:00:00,0,bug
2010-11-02 00:00:00,2010-11-02 00:00:00,0,bug
2010-11-02 01:00:00,2010-11-02 1:00:00,100,Opus
2010-11-02 02:00:00,2010-11-02 2:00:00,0,bug


In [15]:

productsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}} # name of application: molecule component and application dose
interception = InterceptModel(productsDB) # Model of pesitide appplication
interception

In [21]:
# canopy
wheat=echap.get_reconstruction(name="Mercia",nplants=2)
g = wheat.setup_canopy(age=1500)

# pesticide interception
pesticide_interception(g=g, interception_model= interception, application_data=pest_calendar, label='LeafElement')
scene=plot_pesticide(g,colmap=cm.winter_r)
PlantGL(scene,group_by_color=False)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\scipy\interpolate\interpolate.py:446: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = array(y, copy=self.copy)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of li

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

#### Simulation canopy, growth and pesticide interception using weather data

In [22]:
import pandas
from matplotlib import cm

In [24]:
# canopy import
from alinea.echap.architectural_reconstructions import EchapReconstructions

# Weather import
from alinea.echap.weather_data import get_weather

from alinea.alep.mini_models import linear_degree_days
from alinea.alep.protocol import update
from alinea.alep.growth_control import NoPriorityGrowthControl

from alinea.astk.TimeControl import DegreeDayModel, thermal_time_filter, date_filter
from alinea.astk.TimeControl import IterWithDelays , time_control

# Pesticide import
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.interfaces import pesticide_interception
from alinea.echap.tests_nodes import plot_pesticide


In [25]:
# Weather parameter

## import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

##check weather data and add degree day variable using linear degree day model
weather.check(["degree_days",'temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure'],models={'degree_days':linear_degree_days})

## weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", end='2011-05-09', freq='H')  # Range date index. among weather data select weather period for simulation

## canopy timing using degree day
TTmodel = DegreeDayModel(Tbase = 0) # Thermal times model
every_dd = thermal_time_filter(time_sequence=weather_time_index, weather=weather, model=TTmodel, delay = 15) # allows to filter thermal time data here each 15 degree day
canopy_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_dd, data=weather.data)) # create data from weatherdata according to filter

# Pesticide model
# Pesticide data and interception model
application_data = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 100, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 100, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 100, Opus
2010-05-22 11:00:00, 0, bug
"""
productsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}} # name of application: molecule component and application dose
interception = InterceptModel(productsDB) # Model of pesitide appplication

#pest timing
pest_calendar = pesticide_applications(application_data)
every_pest = date_filter(time_sequence=weather_time_index, time_data=pest_calendar)
pest_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_pest, data=pest_calendar))

# growth control
growth_controler = NoPriorityGrowthControl()

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1124: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  return self._get_label(key, axis=axis)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


In [27]:
echap = EchapReconstructions() # load architectural parameter reconstruction with echap data
wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g = wheat.setup_canopy(age=500)

for i,controls in enumerate(zip(canopy_timing, pest_timing)):
    canopy_iter, pest_iter = controls
    
    if canopy_iter.eval:
        print('--', canopy_iter.value.index[-1], '--')
        print('-- update microclimate / canopy --')
        g = wheat.grow(g, canopy_iter.value)
        _=update(g, canopy_iter.dt, growth_controler)
        _=wheat.plot(g)

    if pest_iter.eval:
        print('--', pest_iter.value.index[-1], '--')
        print('-- update microclimate / pesticide --')
        
        _=pesticide_interception(g, interception, pest_iter.value, label='LeafElement')
        _=plot_pesticide(g)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

-- 2010-11-03 09:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-02 00:00:00 --
-- update microclimate / pesticide --
-- 2010-11-02 01:00:00 --
-- update microclimate / pesticide --
-- 2010-11-02 02:00:00 --
-- update microclimate / pesticide --
-- 2010-11-04 10:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-05 12:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-06 20:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-09 03:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-11 13:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-12 17:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-13 18:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-14 23:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-17 13:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-20 01:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-22 17:00:00+00:00 --
-- update microclimate / canopy --
-- 2

# Modele Maladie septo

In [ ]:
from alinea.alep.septo3d_v2 import plugin_septoria

In [ ]:
septoria= plugin_septoria()

wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g = wheat.setup_canopy(age=500)

g = disperse(g, emitter, transporter, "septoria", label='LeafElement')

# pesticide Interception

In [ ]:
import pandas

from alinea.echap.interfaces import pesticide_interception
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.weather_data import get_weather

from alinea.astk.TimeControl import time_filter , rain_filter, thermal_time_filter, DegreeDayModel

### Set weather data and init weather variable

* get wheater data

In [ ]:
weather= get_weather(variety="Mercia")
weather.data

* init weather parameter for pesticide model

In [ ]:
periods = 5 # 5000 pour le cycle complet
seq = pandas.date_range(start = "2010-11-02", periods=periods, freq='H')
TTmodel = DegreeDayModel(Tbase = 150)

every_rain = rain_filter(seq, weather)
every_h = time_filter(seq, delay = 6)
every_dd = thermal_time_filter(seq, weather, TTmodel, delay = 15)

### set application and interception model

* Set application data

In [ ]:
applications = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 1, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 1, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 1, Opus
2010-05-22 11:00:00, 0, bug
"""
pesticide_calendar=pesticide_applications(applications)

ProductsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}}

interception = InterceptModel(ProductsDB)

In [ ]:
for cycle in range(grow_cycles):
    canopy_age = canopy_age
    # update canopy
    new_g = wheat.setup_canopy(age=canopy_age)
    pesticide_interception(new_g,interception_model=interception,application_data=pesticide_calendar,label='LeafElement')
    canopy_age = canopy_age + grow_time
